# Modelos Descritivos - Projeto Final
## Residência de Ciência de Dados - CIN/UFPE

- Lucas Couri (lncc2)
- Mariama Oliveira (mcso)

# 1) Objetivo

Gerar áudio de dígitos aleatórios a partir do dataset de áudio de dígitos MNIST (https://www.kaggle.com/alanchn31/free-spoken-digits), utilizamos 3000 arquivos de áudio para o experimento. Além disso deseja-se verificar se a mesma metodologia pode ser aplicada para músicas, de forma a gerar composições musicais com base em músicas de determinado gênero musical disponíveis no dataset GTZAN (https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification).

# 2) Metodologia

Nesta seção serão exibidos os passos e métodos utilizados para a realização do projeto. Vale notar que, a fim de organização, o código relativo a geração de composições musicais se encontra comentado.

### Bibliotecas e dados

In [2]:
import autoencoder as ae
import train as tr
import preprocess as pp
import soundgenerator as sg
import generate as gn

from soundgenerator import SoundGenerator
from autoencoder import VAE
import pickle
import IPython.display as ipd

### Pré-processamento do dados

O pipeline de pré-processamento dos dados consiste em após carregar os arquivos: aplicar padding caso seja necessário, extrair e salvar os spectogramas e normalizar os dados com Min Max.

In [11]:
#Parametros de amostragem do audio
FRAME_SIZE = 512
HOP_LENGTH = 256
DURATION = 0.74  # em segundos
SAMPLE_RATE = 22050
MONO = True

#Path dos dados
SPECTROGRAMS_PATH = "data/fsdd/spectrograms/"
MIN_MAX_VALUES_SAVE_DIR = "data/fsdd/"
FILES_DIR = "data/recordings/"

In [6]:
#Parametros de amostragem de música
FRAME_SIZE = 1024
HOP_LENGTH = 512
DURATION = 5.94  # em segundos
SAMPLE_RATE = 22050
MONO = True

#Path dos dados
SPECTROGRAMS_PATH = "data/fsdd-jazz/spectrograms/"
MIN_MAX_VALUES_SAVE_DIR = "data/fsdd-jazz/"
FILES_DIR = "data/jazz/"

In [21]:
# Instanciando objetos 
loader = pp.Loader(SAMPLE_RATE, DURATION, MONO)
padder = pp.Padder()
log_spectrogram_extractor = pp.LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
min_max_normaliser = pp.MinMaxNormaliser(0, 1)
saver = pp.Saver(SPECTROGRAMS_PATH, MIN_MAX_VALUES_SAVE_DIR)

preprocessing_pipeline = pp.PreprocessingPipeline()
preprocessing_pipeline.loader = loader
preprocessing_pipeline.padder = padder
preprocessing_pipeline.extractor = log_spectrogram_extractor
preprocessing_pipeline.normaliser = min_max_normaliser
preprocessing_pipeline.saver = saver

preprocessing_pipeline.process(FILES_DIR)

Processed file data/jazz/jazz.00000.wav
Processed file data/jazz/jazz.00001.wav
Processed file data/jazz/jazz.00002.wav
Processed file data/jazz/jazz.00003.wav
Processed file data/jazz/jazz.00004.wav
Processed file data/jazz/jazz.00005.wav
Processed file data/jazz/jazz.00006.wav
Processed file data/jazz/jazz.00007.wav
Processed file data/jazz/jazz.00008.wav
Processed file data/jazz/jazz.00009.wav
Processed file data/jazz/jazz.00010.wav
Processed file data/jazz/jazz.00011.wav
Processed file data/jazz/jazz.00012.wav
Processed file data/jazz/jazz.00013.wav
Processed file data/jazz/jazz.00014.wav
Processed file data/jazz/jazz.00015.wav
Processed file data/jazz/jazz.00016.wav
Processed file data/jazz/jazz.00017.wav
Processed file data/jazz/jazz.00018.wav
Processed file data/jazz/jazz.00019.wav
Processed file data/jazz/jazz.00020.wav
Processed file data/jazz/jazz.00021.wav
Processed file data/jazz/jazz.00022.wav
Processed file data/jazz/jazz.00023.wav
Processed file data/jazz/jazz.00024.wav


### Arquitetura do Variational AutoEncoder

A arquitetura do Variational Autoencoder é composta por 5 camadas convolucionais, sendo cada uma composta por uma conv 2d, uma camada ReLU e uma camada de batch normalization. No meio do encoder/decoder há também uma camada densa onde é feito o cálculo do espaço latente.

### Treinando Modelo

O modelo foi treinado a partir dos espectogramas extraídos anteriormente, eles tem tamanho 256x64.

In [13]:
# Parâmetros para áudios de dígitos
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
EPOCHS = 150
DIR_MODEL = "model/digits/"
#SPECTROGRAMS_PATH = "data/fsdd/spectrograms/"

In [22]:
# Parâmetros para músicas
LEARNING_RATE =  0.0005
BATCH_SIZE = 1
EPOCHS = 6
DIR_MODEL = "model/jazz/"
#SPECTROGRAMS_PATH = "data/fsdd//"

In [14]:
x_train = tr.load_fsdd(SPECTROGRAMS_PATH)
autoencoder = tr.train(x_train, LEARNING_RATE, BATCH_SIZE, EPOCHS)
autoencoder.save(DIR_MODEL)

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 256, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 encoder_conv_layer_1 (Conv2D)  (None, 128, 32, 512  5120        ['encoder_input[0][0]']          
                                )                                                                 
                                                                                                  
 encoder_relu_1 (ReLU)          (None, 128, 32, 512  0           ['encoder_conv_layer_1[0][0]']   
                                )                                                           

KeyboardInterrupt: 

### Reconstruindo amostra a partir Espaço Latente

- Falar da abordagem de pegar uma musica e mapear no espaço

In [15]:
# Diretórios dos áudios dos dígitos
SAVE_DIR_ORIGINAL = "samples/digits/original/"
SAVE_DIR_GENERATED = "samples/digits/generated/"
MIN_MAX_VALUES_PATH = "data/fsdd/min_max_values.pkl"


In [23]:
# Diretórios das músicas
SAVE_DIR_ORIGINAL = "samples/jazz/original/"
SAVE_DIR_GENERATED = "samples/jazz/generated/"
MIN_MAX_VALUES_PATH = "data/fsdd-jazz/min_max_values.pkl"


In [24]:
# Carregar modelo treinado
vae = VAE.load(DIR_MODEL)
sound_generator = SoundGenerator(vae, HOP_LENGTH)

# Carregar espectrograma + valores min max
with open(MIN_MAX_VALUES_PATH, "rb") as f:
    min_max_values = pickle.load(f)

specs, file_paths = gn.load_fsdd(SPECTROGRAMS_PATH)

# sample espectrograma + valores min max
sampled_specs, sampled_min_max_values = gn.select_spectrograms(specs,
                                                            file_paths,
                                                            min_max_values,
                                                            5)

# Gerar audio a partir do espaço latente
signals, _ = sound_generator.generate(sampled_specs,
                                        sampled_min_max_values)

# Convert espectrograma para arquivo de audio
original_signals = sound_generator.convert_spectrograms_to_audio(
    sampled_specs, sampled_min_max_values)

# Salva sinais de audio
gn.save_signals(signals, SAVE_DIR_GENERATED)
gn.save_signals(original_signals, SAVE_DIR_ORIGINAL)

['data/fsdd-jazz/spectrograms/jazz.00017.wav.npy', 'data/fsdd-jazz/spectrograms/jazz.00044.wav.npy', 'data/fsdd-jazz/spectrograms/jazz.00017.wav.npy', 'data/fsdd-jazz/spectrograms/jazz.00076.wav.npy', 'data/fsdd-jazz/spectrograms/jazz.00024.wav.npy']
[{'min': -46.29176, 'max': 33.70824}, {'min': -45.723175, 'max': 34.276825}, {'min': -46.29176, 'max': 33.70824}, {'min': -44.434, 'max': 35.566}, {'min': -48.486347, 'max': 31.513653}]


# 3) Análise de resultados

Para executar os áudios é necessário rodar o notebook utilizando o Jupyter.

### Resultados dos áudios de dígitos

In [11]:
### Audio original 1
audio1_orig = SAVE_DIR_ORIGINAL+"0.wav"
ipd.Audio(audio1_orig, rate=SAMPLE_RATE)

In [12]:
### Audio gerado 1
audio1_gen = SAVE_DIR_GENERATED+"0.wav"
ipd.Audio(audio1_gen, rate=SAMPLE_RATE)

In [13]:
### Audio original 2
audio2_orig = SAVE_DIR_ORIGINAL+"3.wav"
ipd.Audio(audio2_orig, rate=SAMPLE_RATE)

In [14]:
### Audio gerado 2
audio2_gen = SAVE_DIR_GENERATED+"3.wav"
ipd.Audio(audio2_gen, rate=SAMPLE_RATE)

### Resultados das músicas

In [7]:
### Musica original 1
audio1_orig = "samples/jazz/original/0.wav"
ipd.Audio(audio1_orig, rate=SAMPLE_RATE)

In [8]:
### Musica gerado 1
audio1_gen = "samples/jazz/generated/0.wav"
ipd.Audio(audio1_gen, rate=SAMPLE_RATE)

In [9]:
### Musica original 2
audio2_orig = "samples/jazz/original/3.wav"
ipd.Audio(audio2_orig, rate=SAMPLE_RATE)

In [10]:
### Musica gerado 2
audio2_gen = "samples/jazz/generated/3.wav"
ipd.Audio(audio2_gen, rate=SAMPLE_RATE)

# 4) Considerações finais

Comentários gerais:

- A arquitetura funcionou bem para o áudio dos dígitos, conseguindo reproduzir com poucas perdas o mesmo dígito. Já para as músicas notamos que não gerou áudios similares aos originais, isso pode ser devido a vários motivos. 

- Um palpite seria que talvez o uso de uma arquitetura recorrente forneça melhores resultados para este tipo de problema, uma vez que vários trabalhos e artigos da área utilizam redes recorrentes como solução.

- Outra possibilidade é a amostragem do pré-processamento, que tem uma quantidade limitada de amostras, já que música necessita de mais dados do que fala para representar informações. 

- Talvez seja melhor representar as músicas em outro tipo de codificação, mais compacta, como a midi. Isso pode fornecer uma melhor informação sem perda de qualidade. 

- Outra abordagem interessante é investir em técnicas de decomposição da forma de onda para extrair as features mais importantes da música. 

- Vale notar que o dataset dos áudios dos dígitos é bem mais extenso do que o dataset das músicas, além da complexidade de informações dos dois casos ser bastante discrepante.

- Tivemos dificuldade na etapa de treinamento do modelo, devido ao custo computacional dessa tarefa ocasionado pela grande dimensionalidade dos dados. Uma possível abordagem é utilizar técnicas para reduzir previamente a dimensionalidade dos dados.

Vimos que o Convolutional Variational Autoencoder é bastante utilizado para geração de composições musicais, no entanto as arquiteturas que são geralmente utilizadas são mais complexas do que as utilizadas neste projeto. Mas apesar dos resultados ruins para músicas o processo auxiliou no entendimento da arquitetura e seu funcionamento.